In [1]:
import gc
import os
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import keras
from sklearn.model_selection import TimeSeriesSplit

In [2]:
data = pd.read_csv('train.csv').fillna(0).drop(columns=['date_id', 'forward_returns', 'risk_free_rate'])
data

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V13,V2,V3,V4,V5,V6,V7,V8,V9,market_forward_excess_returns
0,0,0,0,1,1,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.003038
1,0,0,0,1,1,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.009114
2,0,0,0,1,0,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.010243
3,0,0,0,1,0,0,0,0,0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004046
4,0,0,0,1,0,0,0,0,0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0,0,0,0,0,0,0,0,0,1.565379,...,-0.432282,0.785053,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,0.001990
8986,0,0,0,0,0,0,0,0,0,1.562946,...,-0.429506,0.767857,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.001845
8987,0,0,1,0,0,0,0,0,0,1.560520,...,-0.425462,0.734127,0.481481,0.787698,0.834898,0.823413,-0.723949,0.133929,-0.670946,0.002424
8988,0,0,0,0,0,0,0,0,0,1.558102,...,-0.385170,0.695106,0.655423,0.783730,0.994026,0.851852,-0.684937,0.101852,-0.646265,0.007843


In [3]:
y = (data['market_forward_excess_returns'] > 0).astype(float)
X = data.drop(columns=['market_forward_excess_returns'])

In [4]:
n_features = len(X.columns) # 94

layers = keras.layers
Sequential = keras.Sequential

models = [
    Sequential([
        layers.Input([n_features]),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features, activation='tanh'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features * 2, activation='tanh'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features * 2, activation='tanh'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features, activation='tanh'),
        layers.Dropout(0.1),
        layers.Dense(n_features * 4, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(n_features * 4, activation='relu'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features, activation='tanh'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features * 2, activation='elu'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(n_features * 2, activation='elu'),
        layers.Dropout(0.1),
        layers.BatchNormalization(),
        layers.Dense(1, activation='elu')
    ], name='v5'),
]

for model in models:
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.MeanAbsoluteError()])


I0000 00:00:1759020124.080910  365986 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1284 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1759020124.081353  365986 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 4657 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5


In [5]:
for k, (i_train, i_val) in enumerate(TimeSeriesSplit().split(X)):
    for i, model in enumerate(models):
        prefix = f'Training model {i+1}/{len(models)}: {model.name} on fold {k+1}'
        print(f'{prefix}...', end='\r')
        model.fit(X.iloc[i_train, :], y.iloc[i_train], epochs=20, validation_data=(X.iloc[i_val, :], y.iloc[i_val]), verbose=0, callbacks=[keras.callbacks.TerminateOnNaN()])
        model.save(f'.models/{model.name}.keras')
        print(f'{prefix}...Complete.')
        while gc.collect() > 0: pass # clear as much memory as possible

I0000 00:00:1759020133.703790  366114 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training model 1/1: v5 on fold 1...Complete.
Training model 1/1: v5 on fold 2...Complete.
Training model 1/1: v5 on fold 3...Complete.
Training model 1/1: v5 on fold 4...Complete.
Training model 1/1: v5 on fold 5...Complete.
